In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization,ReLU
from keras.optimizers import Adam,SGD
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import emoji
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
import string
from keras.utils import to_categorical
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [22]:
nltk.download('stopwords')
nltk.download('punkt')
path = "./sentimentdataset.csv"
df = pd.read_csv(path)

[nltk_data] Downloading package stopwords to /home/ap/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ap/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def preprocess(df):
    df['Sentiment'] = df['Sentiment'].str.strip().str.replace(r'\s+', '', regex=True)
    class_counts = df['Sentiment'].value_counts()
    rare_classes = class_counts[class_counts < 2].index
    balanced_df = df.copy()
    for rare_class in rare_classes:
        rare_samples = df[df['Sentiment'] == rare_class]
        duplicated_samples = resample(rare_samples, replace=True, n_samples=5, random_state=42)
        balanced_df = pd.concat([balanced_df, duplicated_samples], ignore_index=True)
    df = balanced_df
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    def preprocess_text(text):
        text = text.lower()
        text = re.sub(r'http\S+', '', text)
        text = emoji.replace_emoji(text, replace='')
        text = text.translate(str.maketrans("", "", string.punctuation))
        text = re.sub(r"[^a-zA-Z]", " ", text)
        text = re.sub(r"\s+", " ", text).strip()
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word not in stop_words]
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        return " ".join(tokens)
    df1=df.copy()
    df1['clean_text'] = df1['Text'].apply(preprocess_text)
    df1['clean_text'].head()
    df1['label_num'] = df1['Sentiment'].astype('category').cat.codes
    _, X_test_df1, __, y_test_df1 = train_test_split(
            df1['clean_text'], df1['label_num'], test_size=0.2, random_state=42, stratify=df1['label_num']
    )
    vectorizer = TfidfVectorizer(max_features=5000)
    _ = vectorizer.fit_transform(_)
    X_test_tfidf1 = vectorizer.transform(X_test_df1)
    return X_test_tfidf1,y_test_df1

In [24]:
from keras.models import load_model
model1=load_model('./part2_model1.keras')
model2=load_model('./part2_model2.keras')
model3=load_model('./part2_model3.keras')
model4=load_model('./part2_model4.keras')

In [25]:
def predict(model, x_test, y_test):
    y_pred = model.predict(x_test)
    y_test11 = to_categorical(y_test, num_classes=191)
    y_test_labels = np.argmax(y_test11, axis=1)
    y_pred_labels = np.argmax(y_pred, axis=1)
    return accuracy_score(y_test_labels, y_pred_labels)


In [26]:
X_test_tfidf1,y_test_df1=preprocess(df)

In [27]:
print(predict(model1,X_test_tfidf1,y_test_df1),predict(model2,X_test_tfidf1,y_test_df1),predict(model3,X_test_tfidf1,y_test_df1),predict(model4,X_test_tfidf1,y_test_df1))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.6858407079646017 0.6902654867256637 0.7035398230088495 0.7345132743362832
